In [1]:
import xgboost
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import io
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df= pd.read_csv('drive/My Drive/first_preprocessed.csv')

In [3]:
df.shape

(7120, 108)

In [4]:
X= df.drop('price', axis=1)
y= df['price']

In [5]:
X.shape, y.shape

((7120, 107), (7120,))

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=  train_test_split(X,y, test_size=0.2, random_state=51)

In [7]:
print('shapeX_train=',X_train.shape)
print('shapeX_test=',X_test.shape)
print('shapey_train=',y_train.shape)
print('shapeX_test=',y_test.shape)

shapeX_train= (5696, 107)
shapeX_test= (1424, 107)
shapey_train= (5696,)
shapeX_test= (1424,)


In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train= sc.transform(X_train)
X_test = sc.transform(X_test)

In [9]:
import xgboost as xgb

In [10]:
xgb_reg= xgb.XGBRegressor()
xgb_reg.fit(X_train, y_train)

[09:05:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [11]:
xgb_reg_score= xgb_reg.score(X_test, y_test)
xgb_reg_score

0.8759391907485039

In [14]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)
rfr_score=rfr.score(X_test,y_test) # with 0.8863376025408044
rfr_rmse = rmse(y_test, rfr.predict(X_test))
rfr_score, rfr_rmse

(0.8920689110710748, 2168.6525609098126)

In [13]:
from sklearn.metrics import mean_squared_error as rmse


In [15]:
y_pred= xgb_reg.predict(X_test)
y_pred

array([ 83.29294 ,  38.36225 , 119.33976 , ...,  27.489305,  61.979298,
       168.26028 ], dtype=float32)

In [16]:
xgb_reg_rmse= rmse(y_test, y_pred)
xgb_reg_rmse1= np.sqrt(xgb_reg_rmse)
xgb_reg_rmse1

49.92740744112327

In [17]:
from sklearn.model_selection import KFold, cross_val_score
cvs= cross_val_score(xgb_reg, X_train, y_train, cv=10)
cvs, cvs.mean()

[09:07:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:07:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

(array([0.99402902, 0.99012743, 0.99280071, 0.9848235 , 0.98728888,
        0.99348862, 0.98170347, 0.9425467 , 0.99453026, 0.97829862]),
 0.9839637207435251)

In [18]:
list(X.columns)

['bath',
 'balcony',
 'total_sqft_int',
 'bhk',
 'price_per_sqft',
 'area_typeSuper built-up  Area',
 'area_typeBuilt-up  Area',
 'area_typePlot  Area',
 'availability_Ready_To_Move',
 'location_Whitefield',
 'location_Sarjapur  Road',
 'location_Electronic City',
 'location_Haralur Road',
 'location_Raja Rajeshwari Nagar',
 'location_Marathahalli',
 'location_Bannerghatta Road',
 'location_Hennur Road',
 'location_Uttarahalli',
 'location_Thanisandra',
 'location_Electronic City Phase II',
 'location_Hebbal',
 'location_7th Phase JP Nagar',
 'location_Yelahanka',
 'location_Kanakpura Road',
 'location_KR Puram',
 'location_Sarjapur',
 'location_Rajaji Nagar',
 'location_Kasavanhalli',
 'location_Bellandur',
 'location_Begur Road',
 'location_Kothanur',
 'location_Banashankari',
 'location_Hormavu',
 'location_Harlur',
 'location_Akshaya Nagar',
 'location_Electronics City Phase 1',
 'location_Jakkur',
 'location_Varthur',
 'location_HSR Layout',
 'location_Ramamurthy Nagar',
 'locatio

In [19]:
def predict_house_price(model,bath,balcony,total_sqft_int,bhk,price_per_sqft,area_type,availability,location):
  x =np.zeros(len(X.columns))
  x[0]=bath
  x[1]=balcony
  x[2]=total_sqft_int
  x[3]=bhk
  x[4]=price_per_sqft
 
  if "availability"=="Ready To Move":
    x[8]=1

  if 'area_type'+area_type in X.columns:
    area_type_index = np.where(X.columns=="area_type"+area_type)[0][0]
    x[area_type_index] =1

    print(area_type_index)

  if 'location_'+location in X.columns:
    loc_index = np.where(X.columns=="location_"+location)[0][0]
    x[loc_index] =1

  x = sc.transform([x])[0]

  return model.predict([x])[0]

 

In [25]:
predict_house_price(model=rfr, bath=3,balcony=2,total_sqft_int=1672,bhk=3,price_per_sqft=8971.291866,area_type="Plot  Area",availability="Ready To Move",location="Devarabeesana Halli")

7


149.97

In [23]:
import joblib
# save model

joblib.dump(rfr, 'bangalore_house_price_prediction_rfr_model.pkl')

 
# load model
bangalore_house_price_prediction_rfr_model = joblib.load("bangalore_house_price_prediction_rfr_model.pkl")
 

AttributeError: ignored

In [24]:
predict_house_price(bangalore_house_price_prediction_rfr_model,bath=3,balcony=3,total_sqft_int=150,bhk=3,price_per_sqft=8514.285714,area_type="Built-up Area",availability="Ready To Move",location="Devarabeesana Halli")

NameError: ignored